In [ ]:
import sys
sys.path.append("../../.")

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from tqdm.auto import tqdm
from Utils.Utils import loadObjectPickleSnapshot, createObjectPickleSnapshot
from Utils.dica33Utils import getAllHrefForArgument, getTextFromArticle

In [ ]:
options = Options()
options.headless = True 
options.add_argument("--window-size=1280,1024")

In [ ]:
driver = webdriver.Chrome(options=options)
linkSet = set()

linkToSearchList = ["https://www.dica33.it/esperto-risponde/stomaco/", "https://www.dica33.it/esperto-risponde/reflusso/", 
                    "https://www.dica33.it/esperto-risponde/celiachia/", "https://www.dica33.it/esperto-risponde/peso/" ,
                    "https://www.dica33.it/esperto-risponde/domande-stomaco-e-intestino/", "https://www.dica33.it/esperto-risponde/domande-mente-e-cervello/",
                    "https://www.dica33.it/esperto-risponde/domande-scheletro-e-articolazioni/", "https://www.dica33.it/esperto-risponde/domande-fegato/",
                    "https://www.dica33.it/esperto-risponde/domande-pelle/", "https://www.dica33.it/esperto-risponde/domande-cuore-circolazione-e-malattie-del-sangue/",
                    "https://www.dica33.it/esperto-risponde/domande-orecchie-naso-e-gola/", "https://www.dica33.it/esperto-risponde/domande-occhio-e-vista/"
                    ]

for link in tqdm(linkToSearchList):
    linkSet.update(getAllHrefForArgument(driver, link, 70))

In [ ]:
linkSetPath = "dica33LinkSet.pkl"
createObjectPickleSnapshot(linkSet,linkSetPath)

In [7]:
linkSetPath = "dica33LinkSet.pkl"
pickleLinkSet = loadObjectPickleSnapshot(linkSetPath)

In [11]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from queue import Queue

driver_queue = Queue()

def create_driver():
    driver = webdriver.Chrome()  
    driver_queue.put(driver)
    return driver


num_drivers = 10
drivers = [create_driver() for _ in range(num_drivers)]

def process_link(link):
    driver = driver_queue.get()
    try:
        result = getTextFromArticle(driver, link)
    except Exception as e:
        print(f"Error processing link {link}: {str(e)}")
    finally:
        driver_queue.put(driver)
    return (result, link)

with ThreadPoolExecutor(max_workers=num_drivers) as executor:
    results = list(tqdm(executor.map(process_link, pickleLinkSet), total=len(pickleLinkSet)))

  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
import pandas as pd
df = pd.DataFrame(results)
df.to_pickle("dica33DataNotClean.pkl")

In [20]:
df[0][0][0]

["Domande e risposte »\nRisposte di Occhio e vista »\n\n21 giugno 2007\nCataratte urgente!\nMia nonna, 65 enne soffre da 30 anni di artrite reumatoide.\nL'anno scorso gli è comparsa una malattia alla pelle:la pemfigoide bollosa, è stata trattata con flebo di cortisone e le bolle sono scomparse.\nAttualmente a casa continua la cura per la pemfigoide con pillole di cortisone, inoltre ha problemi di Diabete e fa 3 volte al giorno punture di insulina.\nLe scrivo tutto questo prima di parlarle della sua malattia agli occhi ( che abbiamo scoperto oggi tramite visita oculistica ), per farle un pò capire le sue condizioni generali di salute.\nLa diagnosi della visita eseguita agli occhi oggi è stata: cataratte in entrambi gli occhi, per cui la dottoressa ha consigliato di eseguire un operazione col laser per risolvere il problema.\nLei cosa ne pensa considerando che mia nonna ha altre complicanze quali ad esempio il Diabete ?\nE'un intervento rischioso?\nGrazie mille\n\nRisposta del 30 giugno 